In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, ensemble, tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV,cross_validate
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, auc,roc_auc_score,roc_curve,recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib
from mlxtend.plotting import plot_confusion_matrix

import warnings
warnings.filterwarnings("ignore")


import theano
import theano.tensor as Tensor
import random
import sys
import csv
import time
import utils
import visualize
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Bidirectional, LayerNormalization
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import regularizers
import re
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dense, Flatten 

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data4 = pd.read_csv('data4.csv')

In [3]:
data4 = data4.drop(['Unnamed: 0'], axis = 1)

In [4]:
train,test= train_test_split(data4, test_size=0.3,random_state=123, stratify=data4.Result)# stratify the outcome
X_train =train.drop(['Result'], axis = 1)
X_test = test.drop(['Result'], axis = 1)
Y_train = train['Result'].values
Y_test = test['Result'].values

In [5]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Tensorflow/Keras: 2.9.0


In [6]:
X_test = np.expand_dims(X_test, axis=2)

In [7]:
X_test.shape

(395863, 63, 1)

In [8]:
X_train = np.expand_dims(X_train, axis=2)

In [9]:
X_train.shape

(923679, 63, 1)

In [10]:
model = keras.Sequential()
# GRU LAYER IS ADDED TO MODEL WITH 128 CELLS IN IT
model.add(GRU(50, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.2))  # 20% DROPOUT ADDED FOR REGULARIZATION
model.add(BatchNormalization())
model.add(GRU(30, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=True))   # ADD ANOTHER LAYER
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(GRU(15, input_shape=(X_train.shape[1],X_train.shape[2]), activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # FINAL CLASSIFICATION LAYER WITH 2 CLASSES AND SOFTMAX
# ---------------------------------------------------------------------------------------------------
# OPTIMIZER SETTINGS
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

# MODEL COMPILE
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','Recall','Precision'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 63, 50)            7950      
                                                                 
 dropout (Dropout)           (None, 63, 50)            0         
                                                                 
 batch_normalization (BatchN  (None, 63, 50)           200       
 ormalization)                                                   
                                                                 
 gru_1 (GRU)                 (None, 63, 30)            7380      
                                                                 
 dropout_1 (Dropout)         (None, 63, 30)            0         
                                                                 
 batch_normalization_1 (Batc  (None, 63, 30)           120       
 hNormalization)                                        

In [11]:
# RUN THE MODEL
history = model.fit(X_train, Y_train, epochs=10, batch_size=64, verbose=1,
                    validation_data=(X_test, Y_test))

Epoch 1/10
14433/14433 [==============================] - 1261s 87ms/step - loss: 0.4534 - accuracy: 0.8037 - recall: 0.1799 - precision: 0.7024 - val_loss: 0.4312 - val_accuracy: 0.8138 - val_recall: 0.1913 - val_precision: 0.8214
Epoch 2/10
14433/14433 [==============================] - 1246s 86ms/step - loss: 0.3988 - accuracy: 0.8273 - recall: 0.3232 - precision: 0.7434 - val_loss: 0.3494 - val_accuracy: 0.8516 - val_recall: 0.4460 - val_precision: 0.7834
Epoch 3/10
14433/14433 [==============================] - 1945s 135ms/step - loss: 0.3358 - accuracy: 0.8589 - recall: 0.4928 - precision: 0.7826 - val_loss: 0.2725 - val_accuracy: 0.8854 - val_recall: 0.6003 - val_precision: 0.8292
Epoch 4/10
14433/14433 [==============================] - 1709s 118ms/step - loss: 0.2881 - accuracy: 0.8815 - recall: 0.6014 - precision: 0.8088 - val_loss: 0.2322 - val_accuracy: 0.9064 - val_recall: 0.6802 - val_precision: 0.8633
Epoch 5/10
14433/14433 [==============================] - 1950s 135ms/